In [46]:
import scipy.io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize

from scipy.io import loadmat, savemat
import pandas as pd
import scipy.special as SS
import scipy.stats as SSA
import copy
import random
import math
from sklearn.model_selection import ParameterGrid
import os
import numpy.linalg as LA
import gzip

# load pickle module
import pickle

In [47]:
parameter = scipy.io.loadmat('../data/parameterestimate.mat')
alpha = parameter['alpha']
beta = parameter['beta']

/Users/qingyao/anaconda3/envs/geo/lib/python3.11/site-packages/scipy/io/matlab/_mio.py:227: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


## starting date is 2020-02-21

In [48]:
sen_fips = scipy.io.loadmat('../data/countyfips_num.mat')
sen_fips = sen_fips['countyfips_num']
for i in range(3141):
    if (sen_fips[i+1][0] - sen_fips[i][0])<0:
        print(i)
### this data is sorted

In [49]:
beta_mean = np.mean(beta,axis=1)
alpha_mean = np.mean(alpha,axis=1) ## 3142

In [52]:
alpha_mean.shape

(3142, 315)

### there are 3108 fips in the mainland 
### 3142 fips when i do the simulations
### need to compare them and find the intersection

In [50]:
fips_main = pd.read_csv('fips_sorted.cvs')
fips_all = pd.DataFrame(sen_fips,columns=['FIPS'])
mask = fips_all.FIPS.isin(fips_main.FIPS.values)
indices_main = mask[lambda x: x==True].index.tolist()
main_df = fips_all.loc[indices_main]

In [51]:
## select alpha of the main
alpha_mean_main = alpha_mean[indices_main,:]
beta_mean_main = beta_mean[indices_main,:]

In [53]:
# np.savetxt('beta_mean_main.cvs',beta_mean_main)
np.savetxt('alpha_mean_main.cvs',alpha_mean_main)

In [22]:
estimate_R0 = beta_mean_main*5

In [29]:
estimate_R0_t = np.zeros((315,3108))
for i in range(3108):
    estimate_R0_t[:,i] = estimate_R0[i,:]

In [23]:
fips_main['FIPS'].values

array([ 1001,  1003,  1005, ..., 56041, 56043, 56045])

In [32]:
df_estimate_R0 = pd.DataFrame()

In [33]:
df_estimate_R0['Date'] = pd.date_range(start="2020-02-21", periods=315)

In [36]:
df_add = pd.DataFrame(estimate_R0_t,columns=fips_main['FIPS'].values)
df_estimate_R0 = pd.concat([df_estimate_R0,df_add],axis=1)

In [40]:
df_estimate_R0.set_index('Date',inplace=True)

In [45]:
df_estimate_R0.to_csv('estimate_R0_beta.csv')